In [2]:
import pandas
import cv2
import os
from keras_preprocessing.image import ImageDataGenerator
import numpy as np
import scipy


In [3]:
PATH_TO_ORIGINAL_IMAGE_DIRECTORY = "../CharactersPlate"
THRESHOLD = 127
WINDOW_SIZE = 1000

In [4]:
def rename_files(dir_path):
    for filename in os.listdir(dir_path):
        full_path = os.path.join(dir_path, filename)
        name, extension = os.path.splitext(filename)
        new_filename = name[-1] + extension
        new_path = os.path.join(dir_path, new_filename)
        os.rename(full_path, new_path)

#rename_files(PATH_TO_ORIGINAL_IMAGE_DIRECTORY)

In [5]:
def show_image_on_window(image: np.array, window_name: str = "Image") -> None:
    scale = max(image.shape[:2]) / WINDOW_SIZE
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(window_name, int(
        image.shape[1] // scale), int(image.shape[0] // scale))
    cv2.imshow(window_name, image)
    cv2.waitKey(3000)
    cv2.destroyWindow(window_name)

In [6]:
def adapt_image(image):
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, image_binary = cv2.threshold(image, THRESHOLD, 255, cv2.THRESH_BINARY)
    return image_binary/255

In [7]:
def load_images(dir_path):
        images = []
        chars = []
        for file_name in os.listdir(dir_path):
                name, extension = os.path.splitext(file_name)
                if extension == ".png":
                        original_image = cv2.imread(dir_path + "/" + file_name)
                        binary_image = adapt_image(original_image)
                        images.append(binary_image)
                        chars.append(name)
        return np.array(images), np.array(chars)

In [8]:
X, Y = load_images(PATH_TO_ORIGINAL_IMAGE_DIRECTORY)

DATA AUGMENTATION

In [16]:
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.05,
    zoom_range=0.05,
    fill_mode='nearest'
)

In [17]:
X = X.reshape((X.shape[0], X.shape[1], X.shape[2], 1))
datagen.fit(X)

In [18]:
def gen_aug_images(X, y, dir_path, num_aug_img=100):
    tag_count = {}
    
    for image, tag in zip(X, y):
        tag_str = str(tag)
        if tag_str not in tag_count:
            tag_count[tag_str] = 0
        image = np.expand_dims(image, axis=0)
        i = 0
        for batch in datagen.flow(image, batch_size=1):
            tag_count[tag_str] += 1
            filename = f"{tag_str}_{tag_count[tag_str]}.png"
            full_path = os.path.join(dir_path, filename)
            bin_img = (batch[0] > 0.5).astype(np.uint8) * 255
            cv2.imwrite(full_path, bin_img)
            i += 1
            if i >= num_aug_img:
                break 


gen_aug_images(X, Y, PATH_TO_ORIGINAL_IMAGE_DIRECTORY)
